## Implementation of FFT for 2D digital colorful images 

In [ ]:
import numpy as np
import cv2

In [ ]:
def fft2D(image):
    M, N, _ = image.shape
    if M % 2 != 0 or N % 2 != 0:
        raise ValueError("Image dimensions must be even for 2D FFT.")

    # Perform 2D FFT on each color channel
    fft_result = np.zeros_like(image, dtype=np.complex128)
    for c in range(3):
        fft_result[:, :, c] = fft2D_single_channel(image[:, :, c])

    return fft_result


def fft2D_single_channel(channel):
    M, N = channel.shape

    # Perform 1D FFT on rows
    fft_rows = np.zeros_like(channel, dtype=np.complex128)
    for i in range(M):
        fft_rows[i] = fft(channel[i])

    # Perform 1D FFT on columns
    fft_result = np.zeros_like(channel, dtype=np.complex128)
    for j in range(N):
        fft_result[:, j] = fft(fft_rows[:, j])

    return fft_result


def ifft2D(image_fft):
    M, N, _ = image_fft.shape
    if M % 2 != 0 or N % 2 != 0:
        raise ValueError("Image dimensions must be even for 2D IFFT.")

    # Perform 2D IFFT on each color channel
    ifft_result = np.zeros_like(image_fft, dtype=np.complex128)
    for c in range(3):
        ifft_result[:, :, c] = ifft2D_single_channel(image_fft[:, :, c])

    return ifft_result


def ifft2D_single_channel(channel_fft):
    M, N = channel_fft.shape

    # Perform 1D IFFT on rows
    ifft_rows = np.zeros_like(channel_fft, dtype=np.complex128)
    for i in range(M):
        ifft_rows[i] = ifft(channel_fft[i])

    # Perform 1D IFFT on columns
    ifft_result = np.zeros_like(channel_fft, dtype=np.complex128)
    for j in range(N):
        ifft_result[:, j] = ifft(ifft_rows[:, j])

    return ifft_result


def fft(x):
    N = len(x)
    if N <= 1:
        return x

    even = fft(x[0::2])
    odd = fft(x[1::2])
    factor = np.exp(-2j * np.pi * np.arange(N) / N)
    return np.concatenate([even + factor[:N//2] * odd, even + factor[N//2:] * odd])


def ifft(x):
    N = len(x)
    if N <= 1:
        return x

    even = ifft(x[0::2])
    odd = ifft(x[1::2])
    factor = np.exp(2j * np.pi * np.arange(N) / N)
    return np.concatenate([even + factor[:N//2] * odd, even + factor[N//2:] * odd])



## Example of usage FFT for 2D digital colorful  image 
Input is 2D digital colorful image and the output is reconstructed image from frequency by using FFT inverse

In [ ]:
#Example Usage
image = cv2.imread('input.jpg')

#Perform 2D FFT 
fft_result = fft2D(image)

#perform 2D inverse FFT
reconstructed_image = ifft2D(fft_result)
reconstructed_image = np.real(reconstructed_image)

#Scale the pixel values to the range of 0-255
reconstructed_image = (reconstructed_image - np.min(reconstructed_image)) * (255 / np.max(reconstructed_image))
reconstructed_image = reconstructed_image.astype(np.uint8)

#Save the input and output images
#cv2.imwrite("input_image.jpg", image)
#cv2.imwrite("reconstructed_image.jpg", reconstructed_image)

#Display the images 
cv2.imshow("Original Image", image)
cv2.imshow("Reconstructed Image", reconstructed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
